# Task 2: Exploratory Data Analysis

## Objectives
- Analyze patterns in Ethiopia's financial inclusion data
- Visualize Access and Usage trends over time
- Examine event impacts and correlations
- Document key insights and data quality assessment

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Load enriched dataset
df = pd.read_csv('../data/processed/ethiopia_fi_enriched_data.csv')
print(f"Dataset shape: {df.shape}")
print(f"\nRecord types:\n{df['record_type'].value_counts()}")

Dataset shape: (29, 21)

Record types:
record_type
observation    13
event           7
impact_link     7
target          2
Name: count, dtype: int64


In [17]:
# Dataset Overview
print("=== DATASET OVERVIEW ===")
print(f"Total records: {len(df)}")
print(f"Date range: {df[df['record_type'] == 'observation']['observation_date'].min()} to {df[df['record_type'] == 'observation']['observation_date'].max()}")
print(f"\nPillar distribution:\n{df['pillar'].value_counts()}")
print(f"\nConfidence levels:\n{df['confidence'].value_counts()}")
print(f"\nSource types:\n{df['source_name'].value_counts()}")

=== DATASET OVERVIEW ===
Total records: 29
Date range: 2011-12-31 to 2024-12-31

Pillar distribution:
pillar
Access    13
Usage      9
Name: count, dtype: int64

Confidence levels:
confidence
high                                  15
medium                                 6
Telebirr launched by Ethio Telecom     1
Safaricom enters Ethiopian market      1
M-Pesa launches in Ethiopia            1
NFIS-II launched                       1
EthSwitch system upgrade               1
0.15                                   1
0.08                                   1
0.05                                   1
Name: count, dtype: int64

Source types:
source_name
Global Findex 2024                  6
Ethiopia Communication Authority    2
NFIS-II Target                      2
Global Findex 2011                  1
telebirr_launch                     1
National Bank of Ethiopia           1
GSMA Mobile Economy                 1
nfis_ii_launch                      1
mpesa_launch                        1
htt

In [18]:
# Impact Links Analysis
impact_links = df[df['record_type'] == 'impact_link'].copy()

print("=== IMPACT LINKS ANALYSIS ===")
print(f"Total impact links: {len(impact_links)}")
print(f"\nImpact directions:\n{impact_links['impact_direction'].value_counts()}")

# Coerce numeric fields to avoid TypeError when dtype is string
impact_links['impact_magnitude'] = pd.to_numeric(impact_links.get('impact_magnitude', pd.Series(dtype=float)), errors='coerce')
impact_links['lag_months'] = pd.to_numeric(impact_links.get('lag_months', pd.Series(dtype=float)), errors='coerce')

# Safe averages (handle empty or non-numeric)
if impact_links['impact_magnitude'].dropna().empty:
    print("\nAverage impact magnitude: N/A")
else:
    mean_mag = impact_links['impact_magnitude'].dropna().mean()
    print(f"\nAverage impact magnitude: {mean_mag:.3f}")

if impact_links['lag_months'].dropna().empty:
    print("Average lag time: N/A")
else:
    mean_lag = impact_links['lag_months'].dropna().mean()
    print(f"Average lag time: {mean_lag:.1f} months")

# Impact magnitude by pillar (numeric-aware)
impact_by_pillar = impact_links.groupby('pillar')['impact_magnitude'].mean()
print("\nImpact magnitude by pillar:")
for pillar, magnitude in impact_by_pillar.items():
    if pd.isna(magnitude):
        print(f"{pillar}: N/A")
    else:
        print(f"{pillar}: {magnitude:.3f}")

# Event with most impact links
event_impacts = impact_links.groupby('parent_id').size().sort_values(ascending=False)
print("\nEvents with most impact links:")

# prepare events mapping
events = df[df['record_type'] == 'event'].copy()
if 'record_id' in events.columns:
    events['record_id_num'] = pd.to_numeric(events['record_id'], errors='coerce')
else:
    events['record_id_num'] = pd.Series(dtype=float)

for parent_id, count in event_impacts.items():
    pid_num = pd.to_numeric(parent_id, errors='coerce')
    matched = events[events['record_id_num'] == pid_num]
    if not matched.empty:
        event_name = matched['indicator'].iloc[0]
    else:
        event_name = f"Event {parent_id}"
    print(f"{event_name}: {count} impact links")

=== IMPACT LINKS ANALYSIS ===
Total impact links: 7

Impact directions:
impact_direction
positive    4
9.0         1
11.0        1
12.0        1
Name: count, dtype: int64

Average impact magnitude: 0.160
Average lag time: 7.5 months

Impact magnitude by pillar:
Access: 0.200
Usage: 0.120

Events with most impact links:
Event Analyst: 3 impact links
4G Network Launch: 1 impact links
Event 26.0: 1 impact links
Event 27.0: 1 impact links
Event 28.0: 1 impact links


In [34]:
# Gender Gap Analysis
gender_data = observations[observations['indicator_code'].isin(['access_account_male', 'access_account_female'])].copy()
gender_data = gender_data.sort_values(['observation_date', 'indicator_code'])

# Create gender comparison
male_data = gender_data[gender_data['indicator_code'] == 'access_account_male']
female_data = gender_data[gender_data['indicator_code'] == 'access_account_female']

fig = go.Figure()

fig.add_trace(go.Bar(
    x=['2024'],
    y=[male_data['value_numeric'].iloc[0] * 100],
    name='Male',
    marker_color='#1f77b4'
))

fig.add_trace(go.Bar(
    x=['2024'],
    y=[female_data['value_numeric'].iloc[0] * 100],
    name='Female',
    marker_color='#ff7f0e'
))

fig.update_layout(
    title='Gender Gap in Account Ownership (2024)',
    xaxis_title='Year',
    yaxis_title='Account Ownership (%)',
    barmode='group',
    height=400
)

fig.show()

gender_gap = (male_data['value_numeric'].iloc[0] - female_data['value_numeric'].iloc[0]) * 100
print(f"=== GENDER GAP ANALYSIS ===")
print(f"Male account ownership: {male_data['value_numeric'].iloc[0] * 100:.1f}%")
print(f"Female account ownership: {female_data['value_numeric'].iloc[0] * 100:.1f}%")
print(f"Gender gap: {gender_gap:.1f} percentage points")

=== GENDER GAP ANALYSIS ===
Male account ownership: 58.0%
Female account ownership: 40.0%
Gender gap: 18.0 percentage points


In [35]:
# Usage Analysis - Digital Payments and Mobile Money
usage_indicators = ['usage_mm_account', 'usage_digital_payment', 'usage_wages_account']
usage_data = observations[observations['indicator_code'].isin(usage_indicators)].copy()

fig = go.Figure()

colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
for i, indicator in enumerate(usage_indicators):
    data = usage_data[usage_data['indicator_code'] == indicator]
    if not data.empty:
        fig.add_trace(go.Bar(
            x=[data['indicator'].iloc[0]],
            y=[data['value_numeric'].iloc[0] * 100],
            name=data['indicator'].iloc[0],
            marker_color=colors[i]
        ))

fig.update_layout(
    title='Digital Payment Usage Metrics (2024)',
    xaxis_title='Usage Indicator',
    yaxis_title='Percentage of Adults (%)',
    height=500
)

fig.show()

print("=== USAGE ANALYSIS ===")
for _, row in usage_data.iterrows():
    print(f"{row['indicator']}: {row['value_numeric'] * 100:.1f}%")

=== USAGE ANALYSIS ===
Mobile Money Account Ownership: 9.4%
Digital Payment Adoption: 35.0%
Wages Received via Account: 15.0%


In [ ]:
# Infrastructure Analysis
infra_indicators = ['infra_mobile_penetration', 'infra_4g_coverage', 'infra_smartphone_penetration']
infra_data = observations[observations['indicator_code'].isin(infra_indicators)].copy()

fig = go.Figure()

colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
for i, indicator in enumerate(infra_indicators):
    data = infra_data[infra_data['indicator_code'] == indicator]
    if not data.empty:
        fig.add_trace(go.Bar(
            x=[data['indicator'].iloc[0]],
            y=[data['value_numeric'].iloc[0] * 100],
            name=data['indicator'].iloc[0],
            marker_color=colors[i]
        ))

fig.update_layout(
    title='Infrastructure Coverage (2024)',
    xaxis_title='Infrastructure Indicator',
    yaxis_title='Coverage (%)',
    height=500
)

fig.show()

print("=== INFRASTRUCTURE ANALYSIS ===")
for _, row in infra_data.iterrows():
    print(f"{row['indicator']}: {row['value_numeric'] * 100:.1f}%")

# Infrastructure vs Inclusion comparison
print("\n=== INFRASTRUCTURE VS INCLUSION ===")
account_ownership = observations[observations['indicator_code'] == 'access_account_ownership']['value_numeric'].iloc[-1]
mobile_penetration = infra_data[infra_data['indicator_code'] == 'infra_mobile_penetration']['value_numeric'].iloc[0]
smartphone_penetration = infra_data[infra_data['indicator_code'] == 'infra_smartphone_penetration']['value_numeric'].iloc[0]

print(f"Account ownership: {account_ownership * 100:.1f}%")
print(f"Mobile penetration: {mobile_penetration * 100:.1f}%")
print(f"Smartphone penetration: {smartphone_penetration * 100:.1f}%")
print(f"Mobile-to-account ratio: {mobile_penetration/account_ownership:.1f}x")

=== INFRASTRUCTURE ANALYSIS ===
Mobile Penetration: 54.0%
4G Coverage: 35.0%
Smartphone Penetration: 28.0%

=== INFRASTRUCTURE VS INCLUSION ===
Account ownership: 49.0%
Mobile penetration: 54.0%
Smartphone penetration: 28.0%
Mobile-to-account ratio: 1.1x


In [12]:
# Event Timeline and Visual Analysis
events = df[df['record_type'] == 'event'].copy()
# Robust date parsing: coerce errors, fallback to collection_date
events['observation_date_parsed'] = pd.to_datetime(events['observation_date'], errors='coerce')
mask = events['observation_date_parsed'].isna()
if mask.any():
    events.loc[mask, 'observation_date_parsed'] = pd.to_datetime(events.loc[mask, 'collection_date'], errors='coerce')

# Drop events without any parsable date for timeline purposes
valid_events = events[events['observation_date_parsed'].notna()].copy()
valid_events = valid_events.sort_values('observation_date_parsed')

# Create event timeline
fig = go.Figure()

event_colors = {
    'Product Launch': '#1f77b4',
    'Market Entry': '#ff7f0e',
    'Policy': '#2ca02c',
    'Infrastructure': '#d62728'
}

for _, event in valid_events.iterrows():
    category = event['indicator']
    color = event_colors.get(category, '#7f7f7f')
    
    fig.add_trace(go.Scatter(
        x=[event['observation_date_parsed']],
        y=[1],
        mode='markers+text',
        marker=dict(size=15, color=color),
        text=f"{event['indicator']}<br>{event['observation_date_parsed'].strftime('%Y-%m')}",
        textposition='top center',
        name=event['indicator'],
        showlegend=False
    ))

fig.update_layout(
    title='Financial Inclusion Events Timeline (2021-2024)',
    xaxis_title='Date',
    yaxis=dict(showticklabels=False, showgrid=False),
    height=300,
)

fig.show()

print("=== EVENT TIMELINE ===")
for _, event in valid_events.iterrows():
    print(f"{event['observation_date_parsed'].strftime('%Y-%m-%d')}: {event['indicator']} ({event.get('source_name','')})")

/var/folders/wk/8lqwph4j6gd0kmpft5prl_dc0000gn/T/ipykernel_32298/2346525733.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  events['observation_date_parsed'] = pd.to_datetime(events['observation_date'], errors='coerce')


=== EVENT TIMELINE ===
2022-03-15: Mobile Money Regulation (National Bank of Ethiopia)
2023-01-20: 4G Network Launch (Ethio Telecom)


In [36]:
# Overlay Events on Account Ownership Trend
fig = go.Figure()

# Ensure access_data dates are parsed and sorted
access_data['observation_date'] = pd.to_datetime(access_data['observation_date'], errors='coerce')
access_data = access_data.sort_values('observation_date')

# Re-parse events robustly (use observation_date_parsed if available)
events = df[df['record_type'] == 'event'].copy()
if 'observation_date_parsed' not in events.columns:
    events['observation_date_parsed'] = pd.to_datetime(events['observation_date'], errors='coerce')
mask = events['observation_date_parsed'].isna()
if mask.any():
    events.loc[mask, 'observation_date_parsed'] = pd.to_datetime(events.loc[mask, 'collection_date'], errors='coerce')

valid_events = events[events['observation_date_parsed'].notna()].copy()
valid_events = valid_events.sort_values('observation_date_parsed')

# Add account ownership trend
fig.add_trace(go.Scatter(
    x=access_data['observation_date'],
    y=access_data['value_numeric'] * 100,
    mode='lines+markers',
    name='Account Ownership',
    line=dict(width=3),
    marker=dict(size=10)
))

# Add event markers and vertical lines, using nearest account ownership point when available
if not access_data['observation_date'].isna().all() and len(access_data) > 0:
    for _, event in valid_events.iterrows():
        event_date = event['observation_date_parsed']
        # ensure a pandas Timestamp
        if not isinstance(event_date, pd.Timestamp):
            event_date = pd.to_datetime(event_date, errors='coerce')
            if pd.isna(event_date):
                continue

        # compute distances; skip if all NaT
        diffs = (access_data['observation_date'] - event_date).abs()
        if diffs.isna().all():
            continue
        closest_idx = diffs.idxmin()
        closest_value = access_data.loc[closest_idx, 'value_numeric'] * 100

        # vertical line at event date using a shape (safe for datetimes)
        fig.add_shape(
            type='line',
            x0=event_date, x1=event_date,
            y0=0, y1=1,
            xref='x', yref='paper',
            line=dict(color='red', dash='dash')
        )

        # annotation above the plot
        fig.add_annotation(
            x=event_date, y=1.02,
            xref='x', yref='paper',
            text=str(event['indicator']),
            showarrow=False,
            align='center'
        )

        # marker at the nearest data point
        fig.add_trace(go.Scatter(
            x=[event_date],
            y=[closest_value],
            mode='markers+text',
            marker=dict(size=12, color='red'),
            text=[str(event['indicator'])],
            textposition='bottom center',
            showlegend=False
        ))

fig.update_layout(
    title='Account Ownership with Key Events Overlay',
    xaxis_title='Date',
    yaxis_title='Account Ownership (%)',
    height=500,
)

fig.show()


/var/folders/wk/8lqwph4j6gd0kmpft5prl_dc0000gn/T/ipykernel_21299/1104193614.py:11: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



In [37]:
# Correlation Analysis
# Create correlation matrix for available numeric indicators
numeric_data = observations[observations['value_numeric'].notna()].copy()
pivot_data = numeric_data.pivot_table(
    index='observation_date',
    columns='indicator_code',
    values='value_numeric',
    aggfunc='first'
).ffill().bfill()

# Calculate correlations for available indicators
correlation_matrix = pivot_data.corr()

fig = px.imshow(
    correlation_matrix,
    title='Correlation Matrix of Financial Inclusion Indicators',
    color_continuous_scale='RdBu',
    aspect='auto'
)

fig.show()

print("=== CORRELATION ANALYSIS ===")
print("Strong correlations (|r| > 0.5):")
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_val = correlation_matrix.iloc[i, j]
        if abs(corr_val) > 0.5 and not pd.isna(corr_val):
            print(f"{correlation_matrix.columns[i]} ↔ {correlation_matrix.columns[j]}: {corr_val:.3f}")

=== CORRELATION ANALYSIS ===
Strong correlations (|r| > 0.5):


In [38]:
# Impact Links Analysis
impact_links = df[df['record_type'] == 'impact_link'].copy()

print("=== IMPACT LINKS ANALYSIS ===")
print(f"Total impact links: {len(impact_links)}")
print(f"\nImpact directions:\n{impact_links['impact_direction'].value_counts()}")

# Coerce numeric fields to avoid TypeError when dtype is string
impact_links['impact_magnitude'] = pd.to_numeric(impact_links.get('impact_magnitude', pd.Series(dtype=float)), errors='coerce')
impact_links['lag_months'] = pd.to_numeric(impact_links.get('lag_months', pd.Series(dtype=float)), errors='coerce')

# Safe averages (handle empty or non-numeric)
if impact_links['impact_magnitude'].dropna().empty:
    print("\nAverage impact magnitude: N/A")
else:
    mean_mag = impact_links['impact_magnitude'].dropna().mean()
    print(f"\nAverage impact magnitude: {mean_mag:.3f}")

if impact_links['lag_months'].dropna().empty:
    print("Average lag time: N/A")
else:
    mean_lag = impact_links['lag_months'].dropna().mean()
    print(f"Average lag time: {mean_lag:.1f} months")

# Impact magnitude by pillar (numeric-aware)
impact_by_pillar = impact_links.groupby('pillar')['impact_magnitude'].mean()
print("\nImpact magnitude by pillar:")
for pillar, magnitude in impact_by_pillar.items():
    if pd.isna(magnitude):
        print(f"{pillar}: N/A")
    else:
        print(f"{pillar}: {magnitude:.3f}")

# Event with most impact links
event_impacts = impact_links.groupby('parent_id').size().sort_values(ascending=False)
print("\nEvents with most impact links:")

# prepare events mapping
events = df[df['record_type'] == 'event'].copy()
if 'record_id' in events.columns:
    events['record_id_num'] = pd.to_numeric(events['record_id'], errors='coerce')
else:
    events['record_id_num'] = pd.Series(dtype=float)

for parent_id, count in event_impacts.items():
    pid_num = pd.to_numeric(parent_id, errors='coerce')
    matched = events[events['record_id_num'] == pid_num]
    if not matched.empty:
        event_name = matched['indicator'].iloc[0]
    else:
        event_name = f"Event {parent_id}"
    print(f"{event_name}: {count} impact links")

=== IMPACT LINKS ANALYSIS ===
Total impact links: 7

Impact directions:
impact_direction
positive    4
9.0         1
11.0        1
12.0        1
Name: count, dtype: int64

Average impact magnitude: 0.160
Average lag time: 7.5 months

Impact magnitude by pillar:
Access: 0.200
Usage: 0.120

Events with most impact links:
Event Analyst: 3 impact links
4G Network Launch: 1 impact links
Event 26.0: 1 impact links
Event 27.0: 1 impact links
Event 28.0: 1 impact links


## Key Insights Summary

Based on the exploratory data analysis, here are the key findings:

In [ ]:
# Generate Key Insights Summary
insights = [
    "1. Account Ownership Growth Deceleration: Despite massive mobile money expansion (54M+ Telebirr users), account ownership grew only +3pp from 2021-2024, compared to +11pp in the previous period.",
    
    "2. Significant Gender Gap: Women's account ownership (40%) lags men's (58%) by 18 percentage points, indicating gender inclusion challenges.",
    
    "3. Infrastructure-Inclusion Mismatch: Mobile penetration (54%) significantly exceeds account ownership (49%), suggesting barriers beyond basic access.",
    
    "4. Limited Smartphone Adoption: Only 28% smartphone penetration constrains advanced digital payment usage, despite 35% 4G coverage.",
    
    "5. Event Concentration: Major financial inclusion events clustered in 2021-2023 (Telebirr launch, Safaricom entry, M-Pesa launch, 4G rollout).",
    
    "6. Digital Payment Usage Gap: Only 9.45% have mobile money accounts despite 35% reporting digital payment usage, suggesting alternative payment methods."
]

print("=== KEY INSIGHTS ===")
for insight in insights:
    print(f"\n{insight}")

=== KEY INSIGHTS ===

1. Account Ownership Growth Deceleration: Despite massive mobile money expansion (54M+ Telebirr users), account ownership grew only +3pp from 2021-2024, compared to +11pp in the previous period.

2. Significant Gender Gap: Women's account ownership (40%) lags men's (58%) by 18 percentage points, indicating gender inclusion challenges.

3. Infrastructure-Inclusion Mismatch: Mobile penetration (54%) significantly exceeds account ownership (49%), suggesting barriers beyond basic access.

4. Limited Smartphone Adoption: Only 28% smartphone penetration constrains advanced digital payment usage, despite 35% 4G coverage.

5. Event Concentration: Major financial inclusion events clustered in 2021-2023 (Telebirr launch, Safaricom entry, M-Pesa launch, 4G rollout).

6. Digital Payment Usage Gap: Only 9.45% have mobile money accounts despite 35% reporting digital payment usage, suggesting alternative payment methods.


## Data Quality Assessment

In [ ]:
# Data Quality Assessment
print("=== DATA QUALITY ASSESSMENT ===")

# Temporal coverage gaps
years_covered = sorted(observations['year'].unique())
expected_years = list(range(2011, 2025))
missing_years = [year for year in expected_years if year not in years_covered]
print(f"\nTemporal Coverage:")
print(f"Years with data: {years_covered}")
print(f"Missing years: {missing_years}")

# Confidence distribution
confidence_dist = observations['confidence'].value_counts(normalize=True) * 100
print(f"\nConfidence Distribution:")
for conf, pct in confidence_dist.items():
    print(f"{conf}: {pct:.1f}%")

# Data source diversity
source_diversity = observations['source_name'].nunique()
print(f"\nSource Diversity: {source_diversity} unique sources")

# Missing data analysis
missing_data = df.isnull().sum()
high_missing = missing_data[missing_data > len(df) * 0.5]
if len(high_missing) > 0:
    print(f"\nHigh Missingness Fields (>50% missing):")
    for field, count in high_missing.items():
        print(f"{field}: {count} ({count/len(df)*100:.1f}%)")
else:
    print(f"\nNo fields with >50% missing data")

# Data limitations
limitations = [
    "Limited temporal granularity: Most observations are annual, missing quarterly/monthly trends",
    "Sparse infrastructure data: Only 2024 infrastructure metrics available",
    "Regional aggregation: No state/regional disaggregation for geographic analysis",
    "Impact estimation: Impact links based on regional comparators rather than local evidence",
    "Mobile money vs account ownership gap: Unclear why 54M+ mobile money accounts don't translate to higher account ownership"
]

print(f"\n=== DATA LIMITATIONS ===")
for i, limitation in enumerate(limitations, 1):
    print(f"{i}. {limitation}")

=== DATA QUALITY ASSESSMENT ===

Temporal Coverage:
Years with data: [np.int32(2011), np.int32(2014), np.int32(2017), np.int32(2021), np.int32(2024)]
Missing years: [2012, 2013, 2015, 2016, 2018, 2019, 2020, 2022, 2023]

Confidence Distribution:
high: 69.2%
medium: 30.8%

Source Diversity: 7 unique sources

High Missingness Fields (>50% missing):
value_text: 24 (82.8%)
parent_id: 22 (75.9%)
related_indicator: 22 (75.9%)
impact_direction: 22 (75.9%)
impact_magnitude: 22 (75.9%)
lag_months: 22 (75.9%)
evidence_basis: 22 (75.9%)

=== DATA LIMITATIONS ===
1. Limited temporal granularity: Most observations are annual, missing quarterly/monthly trends
2. Sparse infrastructure data: Only 2024 infrastructure metrics available
3. Regional aggregation: No state/regional disaggregation for geographic analysis
4. Impact estimation: Impact links based on regional comparators rather than local evidence
5. Mobile money vs account ownership gap: Unclear why 54M+ mobile money accounts don't translate t